In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-16 20:47:46--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.68MB/s    in 0.6s    

2022-04-16 20:47:48 (1.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [15]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### Create DataFrames to match tables

In [25]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [27]:
# Clean the DataFrame
print(df.count())
# Drop null values
drops_df = df.dropna()
print(drops_df.count())
# Drop null values
clean_df = drops_df.drop_duplicates()
print(clean_df.count())

1785997
1785886
1785886


In [38]:
# Create the customers_table DataFrame
customers_df = clean_df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   40504830|             8|
|    3675423|             3|
|   11393715|             9|
|   14175902|            22|
|   39664150|             4|
|   14534785|             1|
|   47765241|             7|
|   33929721|             1|
|   41446882|             8|
|   52714119|             4|
|   27910865|             1|
|   31564043|            24|
|   50722423|             2|
|   26206997|             1|
|   21130353|             1|
|   28184372|             1|
|   13005988|            16|
|   13673487|             1|
|   16100357|             1|
|   16863015|             1|
+-----------+--------------+
only showing top 20 rows



In [41]:
# Create the products_table DataFrame and drop duplicates. 
products_df = clean_df.select("product_id", "product_title").drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0439394422|school house rock...|
|9861064222|Black Nintendo Wi...|
|B00000K1XN|        WWF Attitude|
|B00001OX40|   Neo Cherry Master|
|B00001QEQD|     USCF Chess - PC|
|B00001XDTO|        Yoda Stories|
|B00001ZWQ0|Legend of the Riv...|
|B0000296ZL|      Game Boy Color|
|B00002SFMS|     Boss Rally - PC|
|B00002SSYX|Ristar (Sega Game...|
|B00002STA7|    NFL Football '94|
|B00002STH3|           Extreme G|
|B00002STJC|                 MDK|
|B00002STWV|Virtual Open Tenn...|
|B00002SW1K|        World Heroes|
|B00002SW49|Lawnmower Man - S...|
|B000035XH1|RoboCop vs. Termi...|
|B000035XYM|Alien vs. Predato...|
|B000035Y3U|Top Gear 2 - Nint...|
|B000046S3Z|           Crystalis|
+----------+--------------------+
only showing top 20 rows



In [44]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df =clean_df.select(["review_id","customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()
review_id_df.printSchema()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10010D6B1QM7U|   26197704|B004HX5OFW|       8155204| 2013-03-28|
|R100BI61LIKH2N|   10409381|B000QW9D14|     583724133| 2013-10-28|
|R100TPJCYDYGBU|   39071377|B00D3IKP7Y|     378642975| 2013-12-31|
|R1018OO9U6UUWO|   32826337|B0094X28J0|     272196441| 2014-03-21|
|R102YIWZEIAXT9|   16385120|B000UQAUWW|     694091073| 2007-11-29|
|R1034S7FA21OI1|   14198390|B004S5PBM0|     275810674| 2011-11-06|
|R103LC8CTAHWZ6|   16965219|B002EE7OKE|     852505908| 2013-10-07|
|R103PT7S4HRIP8|   48295356|B003R7H5TC|     318359987| 2010-12-28|
|R104RCQZYPYDXS|   21497984|B000R3BNDI|     318965716| 2008-02-28|
|R105KZIXHZW63W|   22595160|B000NKFEZI|     256274221| 2013-02-04|
|R105W3ID5NHQ48|   43562425|B002I0JNK8|     930811126| 2011-07-07|
|R106J0FROF1O34|   50248806|B0050SWUTQ|     861146594| 2014-03

In [45]:
# Create the vine_table. DataFrame
vine_df = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R10010D6B1QM7U|          1|            0|          0|   N|                Y|
|R100BI61LIKH2N|          5|            1|          2|   N|                Y|
|R100TPJCYDYGBU|          3|            1|          2|   N|                Y|
|R1018OO9U6UUWO|          5|            0|          0|   N|                Y|
|R102YIWZEIAXT9|          5|            2|          4|   N|                N|
|R1034S7FA21OI1|          5|            2|          2|   N|                Y|
|R103LC8CTAHWZ6|          2|            1|          1|   N|                Y|
|R103PT7S4HRIP8|          2|            0|          0|   N|                Y|
|R104RCQZYPYDXS|          5|            7|          8|   N|                N|
|R105KZIXHZW63W|          5|            1|          1|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [46]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<insert endpoint>:5432/challenge_database"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [47]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [48]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [49]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [50]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)